# Manifest generator for MNIST-like : real vs complex

Train for $40-75-40$ with various kl-div coefficients and the following lr-scheduling scheme:
* `1e-3` for epochs 0..9, `1e-4` for `10+` (as in Trabelsi et al.)


**NB** we use batch averaged target loss and **do not** multiply
by the number of samples in the dataset (for fp arithmetic stability)
-- hence in the sum kl-div case the largest coefficient should be
$\tfrac1{\text{n_samples}}$

$$
\frac1N \widehat{\mathcal{L}}_{\text{sgvb}}
    = \widehat{\mathbb{E}}_{z\sim B}
    \mathbb{E}_{\omega \sim q_\omega} \log p(z\mid \omega)
    - \frac1N \mathbb{E}_{\omega \sim q_\omega} \log\frac{q_\omega(\omega)}{\pi(\omega)}
\,. $$

layer-size averaged ELBO undercompresses and is inconsistent with theory:
$$
    \sum_i \mathbb{E}_{W \sim q_W} \log p(z_i\,\mid W)
    - \lambda \sum_k \frac{C}{\lvert W_k\rvert}
        \sum_{w\in W_k} \mathbb{E}_{w\sim q_w} \log \frac{q_w(w)}{\pi_w(w)}
\,. $$

Sum is consistent and
$$
    \sum_i \mathbb{E}_{W \sim q_W} \log p(z_i\,\mid W)
    - \lambda C \sum_k \sum_{w\in W_k} \mathbb{E}_{w\sim q_w} \log \frac{q_w(w)}{\pi_w(w)}
\,.$$

* Mnist-like dataset:
  * MNIST, KMNIST, Fashion MNIST, EMNIST-Letters
    * emnist has 26 outputs, others - 10

* Feature-model
  * real-real: Raw, `.real.*` with `n_inputs=1`
  * cplx-real: Fourier, `.real.*` with `n_inputs=2`

  * real-cplx: Raw, `.cplx.*` with `n_inputs=1` needs to `upcast real to Cplx`
  * cplx-cplx: Fourier, `.cplx.*` with `n_inputs=1`


In [ ]:
import os
import copy
import numpy as np

from itertools import product

Load default config

In [ ]:
import json
from pkg_resources import resource_stream

with resource_stream("cplxpaper.mnist", "template.json") as fin:
    options = json.load(fin)

Sample some random seeds for train splits.

In [ ]:
np.random.randint(0x7fff_ffff, size=(13,))

No `123`, `0xdeadc0de` or `42` bullshit!
Pick opaque random seed from `np.randint` above.

In [ ]:
from cplxpaper.mnist import dataset

dataset_variety = {
    "mnist": {
        "datasets__train__random_state": [1_641_730_374],
        "datasets__train__cls": [str(dataset.MNIST_Train)],
        "datasets__test__cls": [str(dataset.MNIST_Test)],
        "model__n_outputs": [10],
    },
    "kmnist": {
        "datasets__train__random_state": [102_048_205],
        "datasets__train__cls": [str(dataset.KMNIST_Train)],
        "datasets__test__cls": [str(dataset.KMNIST_Test)],
        "model__n_outputs": [10],
    },
    "fashion-mnist": {
        "datasets__train__random_state": [1_526_761_432],
        "datasets__train__cls": [str(dataset.FashionMNIST_Train)],
        "datasets__test__cls": [str(dataset.FashionMNIST_Test)],
        "model__n_outputs": [10],
    },
    "emnist-letters": {
        "datasets__train__random_state": [605_446_338],
        "datasets__train__cls": [str(dataset.EMNIST_Letters_Train)],
        "datasets__test__cls": [str(dataset.EMNIST_Letters_Test)],
        "model__n_outputs": [26],
    },
}

Enumerate all possbile model `combinations`

In [ ]:
from cplxpaper.mnist import models
from itertools import product, chain

model_combinations = {
    "real": [*chain(product([
            models.real.SimpleConvModel
        ], [
            models.real.SimpleConvModelARD, models.real.SimpleConvModelVD
        ], [
            models.real.SimpleConvModelMasked
        ]), product([
            models.real.TwoLayerDenseModel
        ], [
            models.real.TwoLayerDenseModelARD, models.real.TwoLayerDenseModelVD
        ], [
            models.real.TwoLayerDenseModelMasked
        ])
    )],
    "complex": [*chain(product([
            models.complex.SimpleConvModel
        ], [
            models.complex.SimpleConvModelARD, models.complex.SimpleConvModelVD
        ], [
            models.complex.SimpleConvModelMasked
        ]), product([
            models.complex.TwoLayerDenseModel
        ], [
            models.complex.TwoLayerDenseModelARD, models.complex.TwoLayerDenseModelVD
        ], [
            models.complex.TwoLayerDenseModelMasked
        ])
    )],
}

In [ ]:
model_variety = {}

for name, combinations in model_combinations.items():
    for models in combinations:
        m_dense, m_sparsify, m_masked = map(str, models)
        model_variety.setdefault(name, []).append({
            "stages__dense__model__cls": [m_dense],
            "stages__sparsify__model__cls": [m_sparsify],
            "stages__fine-tune__model__cls": [m_masked],
        })

Update the template with correct data specification

In [ ]:
options.update({
    "datasets": {
        "train": {
            "cls": None,
            "root": '/home/ivan.nazarov/Github/complex_paper/experiments/mnist-like/data',
            "random_state": None,

            # by default use 10k train sample size
            "train_size": 10000
        },
        "test": {
            "cls": None,
            "root": '/home/ivan.nazarov/Github/complex_paper/experiments/mnist-like/data'
        },
    },
    'features': {
        "cls": None
    },
    "feeds": {
        'train': {
            'cls': "<class 'torch.utils.data.dataloader.DataLoader'>",
            'dataset': 'train',
            'batch_size': 128,
            'shuffle': True,
            'pin_memory': False,
            'n_batches': -1
        },
        'test': {
            'cls': "<class 'torch.utils.data.dataloader.DataLoader'>",
            'dataset': 'test',
            'batch_size': 128,
            'shuffle': False,
            'pin_memory': False,
            'n_batches': -1
        }
    },
    "scorers": {},  # we shall score models when building a report
})

Prepare the main template:
* reset roots
* clear model definitions
* specify restarts and grad clips

In [ ]:
from cplxpaper.auto.parameter_grid import get_params, set_params, special_params

options = set_params(options, **{
    "threshold": -0.5,  # we use -1/2 for reasons outlined in the text

    "objective_terms__kl_div__reduction": "sum",
    "objective_terms__kl_div__coef": 1e-4,  # 1 / 10k

    # specify state inheritance
    "stages__sparsify__restart": False,
    "stages__sparsify__reset": False,

    "stages__fine-tune__restart": True,  # restart the optimizer
    "stages__fine-tune__reset": False,

    # L2 clip gradients: seems to be always better to do so.
    "stages__dense__grad_clip": 0.5,
    "stages__sparsify__grad_clip": 0.5,
    "stages__fine-tune__grad_clip": 0.5,

    # train 40-75-40
    "stages__dense__n_epochs": 40,
    "stages__sparsify__n_epochs": 75,
    "stages__fine-tune__n_epochs": 40,

# radish-on-a-plate is decreasing the lr too fast
#     "stages__dense__lr_scheduler__cls": "<class 'torch.optim.lr_scheduler.ReduceLROnPlateau'>",
#     "stages__sparsify__lr_scheduler__cls": "<class 'torch.optim.lr_scheduler.ReduceLROnPlateau'>",
#     "stages__fine-tune__lr_scheduler__cls": "<class 'torch.optim.lr_scheduler.ReduceLROnPlateau'>",

    # in the original run I used this scheduler and it yielded the compression results
    "stages__dense__lr_scheduler__cls": "<class 'cplxpaper.musicnet.lr_scheduler.Trabelsi2017LRSchedule'>",
    "stages__sparsify__lr_scheduler__cls": "<class 'cplxpaper.musicnet.lr_scheduler.Trabelsi2017LRSchedule'>",
    "stages__fine-tune__lr_scheduler__cls": "<class 'cplxpaper.musicnet.lr_scheduler.Trabelsi2017LRSchedule'>",

    # clean models
    "model": {},
    "stages__dense__model": {},
    "stages__sparsify__model": {},
    "stages__fine-tune__model": {},
})

<br>

KL-divergence coeffcient $C$ settings -- directly affects sparsification.

In [ ]:
base_grid = {
    # binary ratio grid scaled by \tfrac3{2^k} for k = -19..-2
    "stages__sparsify__objective__kl_div":
        1.5 * np.logspace(-19, -1, base=2, num=37),

    # use only one threshold for higher sparsity at the cost of worse performance
    "threshold": [-0.5],
}

# np.unique(np.r_[
#     1.0 * np.logspace(-7, -2, 6),
#     3.3 * np.logspace(-7, -2, 6),
#     6.6 * np.logspace(-7, -2, 6),
#     1.0 * np.linspace(0.1, 1, 10)
# ])

The grid

In [ ]:
grid, tag = [], "real-vs-cplx"

### Halving-doubling: Raw

Use raw feaures and compare $\mathbb{R}$ against $\tfrac12 \mathbb{C}$, and $\mathbb{C}$

In [ ]:
from cplxpaper.auto import feeds

features = {
    "features__cls": [str(feeds.FeedRawFeatures),],
    "model__n_inputs": [1],
}
for data, data_options in dataset_variety.items():
    for model_stages in model_variety["real"]:
        grid.append({
            "____name__": [f"Full real-valued model on raw {data} 10k"],
            **data_options,
            **model_stages,
            **features,
            "model__double": [False],
            **base_grid
        })

    for model_stages in model_variety["complex"]:
        grid.append({
            "____name__": [f"Halved complex-valued model on raw {data} 10k"],
            **data_options,
            **model_stages,
            **features,
            "model__half": [True],
            "model__upcast": [True],
            **base_grid
        })

    for model_stages in model_variety["complex"]:
        grid.append({
            "____name__": [f"Full complex-valued model on raw {data} 10k"],
            **data_options,
            **model_stages,
            **features,
            "model__half": [False],
            "model__upcast": [True],
            **base_grid
        })

### Halving-doubling: FFT

Use Fourier feaures and compare $2 \mathbb{R}$ against $\mathbb{C}$, and $\mathbb{R}$

In [ ]:
from cplxpaper.auto import feeds

features = {
    "features__cls": [str(feeds.FeedFourierFeatures),],
    "features__cplx": [True],
    "features__shift": [True],
    "features__signal_ndim": [2],
}
for data, data_options in dataset_variety.items():
    for model_stages in model_variety["real"]:
        grid.append({
            "____name__": [f"Doubled real-valued model on Fourier features of {data} 10k"],
            **data_options,
            **model_stages,
            **features,
            "model__n_inputs": [2],
            "model__double": [True],
            **base_grid
        })

    for model_stages in model_variety["complex"]:
        grid.append({
            "____name__": [f"Full complex-valued model on Fourier features of {data} 10k"],
            **data_options,
            **model_stages,
            **features,
            "model__half": [False],
            "model__upcast": [False],
            **base_grid
        })

    for model_stages in model_variety["real"]:
        grid.append({
            "____name__": [f"Full real-valued model on Fourier features of {data} 10k"],
            **data_options,
            **model_stages,
            **features,
            "model__n_inputs": [2],
            "model__double": [False],
            **base_grid
        })

<br>

## Create the grid

Cerate a master folder to house all grid replications.

In [ ]:
SUFFIX = "-20200213"

base_folder = os.path.normpath(os.path.abspath(f"./grids{SUFFIX}/"))
os.makedirs(base_folder, exist_ok=False)

assert os.path.exists(base_folder) and os.path.isdir(base_folder)

Repeat the experiment 5 times

In [ ]:
n_replications = 5

Write experiment manifest en-masse: put each replication in a separate folder

In [ ]:
import tqdm
from sklearn.model_selection import ParameterGrid

folders = []
for replication in range(n_replications):
    folder = os.path.join(base_folder, f"mnist-like__{tag}__{replication:02d}")
    os.makedirs(folder, exist_ok=False)

    for exp_no, par in enumerate(tqdm.tqdm(ParameterGrid(grid))):
        par, special = special_params(**par)

        local = set_params(copy.deepcopy(options), **par, device=None)
        local.update(special)

        # format the name
        filename = os.path.join(folder, f"experiment__{exp_no:05d}.json")
        json.dump(local, open(filename, "w"), indent=2)
    
    folders.append(folder)

Create a bash script for this grid

In [ ]:
import stat

devspec = """--devices "cuda:0" "cuda:1" "cuda:2" "cuda:3" --per-device 8"""
cmd_report = f'''python -m cplxpaper.auto.reports {devspec} {{kind}} {{output}} {{paths}}'''

bash = os.path.join(base_folder, "mnist-like.sh")
with open(bash, "w") as fout:
    # experiment execution
    for folder in map(os.path.abspath, folders):
        fout.write(f"""python -m cplxpaper.auto {devspec} "{folder}"\n""")

    # report analysis
    paths = '" "'.join(map(os.path.abspath, folders))
    output = os.path.join(base_folder, "report__trade-off.pk")
    fout.write(f"""python -m cplxpaper.auto.reports {devspec} "trade-off" "{output}" "{paths}"\n""")

# allow exc and keep r/w
os.chmod(bash, stat.S_IXUSR | stat.S_IRUSR | stat.S_IWUSR)

bash

In [ ]:
assert False

<br>